In [ ]:
import numpy as np
import pandas as pd

train_set = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')
test_set = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/test.csv')

train_set = train_set.drop('id', axis=1)

In [ ]:
def string_kernel(a, b):
    a_word = a.split()
    b_word = b.split()
    cnt = 0
    for i in range(len(a_word)):
        for j in range(len(b_word)):
            if a_word[i] == b_word[j] or b_word[j] in a_word[i] or a_word[i] in b_word[j]:
                cnt += 1
    result = cnt / (len(a_word) + len(b_word))
    return result

def convert(s):
    r = 0
    r += (ord(s[0]) - ord('A')) * 100
    r += (ord(s[1]) - ord('0')) * 10
    r += ord(s[2]) - ord('0')
    return r

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

y_train = train_set['score']
x_train = train_set.drop('score', axis=1)

context = x_train['context']
label_context = []
for string in context:
    sc = convert(string)
    label_context.append(sc)
x_train['context'] = label_context

relation = []
for i in range(len(y_train)):
    r = string_kernel(x_train['anchor'][i], x_train['target'][i])
    relation.append(r)

x_train_fine = x_train.drop(['anchor', 'target'], axis=1)
x_train_fine['relation'] = relation

x_train_scaled = scaler.fit_transform(x_train_fine)

In [ ]:
from sklearn.model_selection import train_test_split

x_tr, x_te, y_tr, y_te = train_test_split(x_train_scaled, y_train, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

for_reg = RandomForestRegressor(random_state=42, max_depth=10, max_leaf_nodes=120)
for_reg.fit(x_tr, y_tr)

In [ ]:
from sklearn.metrics import mean_absolute_error
pre = for_reg.predict(x_te)
mean_absolute_error(pre, y_te)

In [ ]:
id_test = test_set['id']
x_test = test_set.drop('id', axis=1)
context_t = x_test['context']
label_context_t = []
for string in context_t:
    sc = convert(string)
    label_context_t.append(sc)
x_test['context'] = label_context_t

relation = []
for i in range(len(id_test)):
    r = string_kernel(x_test['anchor'][i], x_test['target'][i])
    relation.append(r)

x_test_fine = x_test.drop(['anchor', 'target'], axis=1)
x_test_fine['relation'] = relation

x_test_scaled = scaler.fit_transform(x_test_fine)

In [ ]:
pre_test = for_reg.predict(x_test_scaled)

In [ ]:
output = pd.DataFrame()
output['id'] = id_test
output['score'] = pre_test
output.to_csv('submission.csv', index=False)